# Notebook 4: Assessment and Evaluation
## Introduction
# Loads models/data from Notebook 3, evaluates on test set with MAE/RMSE/AQI metrics, XAI via SHAP.
# Justification: MAE/RMSE for regression accuracy; weighted F1 for imbalanced AQI classes. SHAP for interpretability in sustainability apps.


In [2]:
# Mount Google Drive
from google.colab import drive
import os

# Mount your Google Drive
drive.mount('/content/drive')

# Define your project folder in Google Drive
your_project_path = '/content/drive/My Drive/AI_Sustainability_Project_lsa'

# Create the project directory if it doesn't exist
os.makedirs(your_project_path, exist_ok=True)
print(f"Project path set to: {your_project_path}")

# Change current working directory to your project path
%cd "{your_project_path}"

# Verify current working directory
!pwd
!ls

Current working directory: /Users/psy/cs/ai/sustain/code
Available files:

Checking /Users/psy/cs/ai/sustain for data files:
  - /Users/psy/cs/ai/sustain/.venv/lib/python3.13/site-packages/numpy/_core/tests/data/umath-validation-set-log2.csv
  - /Users/psy/cs/ai/sustain/.venv/lib/python3.13/site-packages/numpy/_core/tests/data/umath-validation-set-arcsinh.csv
  - /Users/psy/cs/ai/sustain/.venv/lib/python3.13/site-packages/numpy/_core/tests/data/umath-validation-set-arctanh.csv
  - /Users/psy/cs/ai/sustain/.venv/lib/python3.13/site-packages/numpy/_core/tests/data/umath-validation-set-sin.csv
  - /Users/psy/cs/ai/sustain/.venv/lib/python3.13/site-packages/numpy/_core/tests/data/umath-validation-set-cos.csv
  - /Users/psy/cs/ai/sustain/.venv/lib/python3.13/site-packages/numpy/_core/tests/data/umath-validation-set-cbrt.csv
  - /Users/psy/cs/ai/sustain/.venv/lib/python3.13/site-packages/numpy/_core/tests/data/umath-validation-set-arctan.csv
  - /Users/psy/cs/ai/sustain/.venv/lib/python3.13/

In [4]:
# Install required packages
!pip install scikit-learn tensorflow shap seaborn matplotlib pandas numpy joblib

print("All packages installed successfully!")

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 546.8/546.8 kB 17.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 546.8/546.8 kB 17.5 MB/s  0:00:00
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/8.1 MB ? eta -:--:--Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 17.2 MB/s  0:00:00 16.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 17.2 MB/s  0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.8 MB ? eta -:--

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, classification_report, confusion_matrix
import joblib
from tensorflow.keras.models import load_model
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import os
import warnings
warnings.filterwarnings('ignore')

print("--- Starting Assessment and Evaluation (Notebook 4) ---")

# GOOGLE COLAB PATH CONFIGURATION
base_path = '/content/drive/MyDrive/AI_Sustainability_Project_lsa'
print(f"Working directory: {base_path}")

# Check if we're in Google Colab by looking for the mounted drive
if os.path.exists('/content/drive/MyDrive'):
    print("🔥 Running in GOOGLE COLAB environment")
    data_path = base_path
else:
    print("🔥 Running in LOCAL environment - adapting paths")
    data_path = '/Users/psy/cs/ai/sustain/code'
    base_path = data_path

print(f"Data path set to: {data_path}")

# CRITICAL FIX: Check if we have any data at all
print("Checking for available data files...")
try:
    available_files = os.listdir(data_path)
    csv_files = [f for f in available_files if f.endswith('.csv')]
    model_files = [f for f in available_files if f.endswith('.pkl') or f.endswith('.keras')]
    
    print(f"Available CSV files: {csv_files}")
    print(f"Available model files: {model_files}")
except Exception as e:
    print(f"Error listing files: {e}")
    csv_files = []
    model_files = []

# Try to load test data - with fallback options
test_data_loaded = False
df_test = None

# Option 1: Try to load from Notebook 3 output (PREFERRED)
test_data_files = ['test_featured_data.csv', 'test_data.csv']
for test_filename in test_data_files:
    test_filepath = os.path.join(data_path, test_filename)
    if os.path.exists(test_filepath):
        try:
            df_test = pd.read_csv(test_filepath, index_col=0, parse_dates=True)
            print(f"✅ Loaded test data from {test_filename}. Shape: {df_test.shape}")
            print(f"PM2.5 variance: {df_test['pm25_value'].var():.6f}")
            print(f"PM2.5 range: {df_test['pm25_value'].min():.2f} to {df_test['pm25_value'].max():.2f}")
            test_data_loaded = True
            break
        except Exception as e:
            print(f"❌ Failed to load {test_filename}: {e}")

# Option 2: Try to load any CSV with PM2.5 data
if not test_data_loaded and csv_files:
    for csv_file in csv_files:
        try:
            csv_filepath = os.path.join(data_path, csv_file)
            temp_df = pd.read_csv(csv_filepath)
            if 'pm25_value' in temp_df.columns:
                # Try to set timestamp as index
                if 'timestamp' in temp_df.columns:
                    temp_df['timestamp'] = pd.to_datetime(temp_df['timestamp'])
                    temp_df.set_index('timestamp', inplace=True)
                elif temp_df.index.name == 'timestamp':
                    temp_df.index = pd.to_datetime(temp_df.index)
                else:
                    # Create a dummy timestamp index
                    temp_df.index = pd.date_range(start='2020-01-01', periods=len(temp_df), freq='h')
                
                df_test = temp_df
                print(f"✅ Loaded PM2.5 data from {csv_file}. Shape: {df_test.shape}")
                print(f"PM2.5 variance: {df_test['pm25_value'].var():.6f}")
                test_data_loaded = True
                break
        except Exception as e:
            print(f"❌ Failed to load {csv_file}: {e}")

# Option 3: Create synthetic test data if nothing available (LAST RESORT)
if not test_data_loaded:
    print("⚠️  No suitable data found. Creating synthetic PM2.5 data for demonstration...")
    
    # Create realistic PM2.5 time series data with HIGH VARIANCE
    np.random.seed(42)
    n_hours = 1000
    
    # Base PM2.5 level with strong daily and weekly patterns
    hours = np.arange(n_hours)
    daily_pattern = 20 * np.sin(2 * np.pi * hours / 24)  # Strong daily cycle
    weekly_pattern = 10 * np.sin(2 * np.pi * hours / (24 * 7))  # Weekly cycle
    trend = 0.02 * hours  # Slight upward trend
    noise = np.random.normal(0, 12, n_hours)  # Strong random noise
    
    pm25_base = 35 + daily_pattern + weekly_pattern + trend + noise
    pm25_base = np.clip(pm25_base, 5, 150)  # Realistic PM2.5 range
    
    # Create weather features with correlation to PM2.5
    temp = 25 + 8 * np.sin(2 * np.pi * hours / 24) + np.random.normal(0, 3, n_hours)
    humidity = 65 + 15 * np.sin(2 * np.pi * hours / 24 + np.pi) + np.random.normal(0, 8, n_hours)
    humidity = np.clip(humidity, 30, 95)
    wind_speed = 3 + 2 * np.random.exponential(1, n_hours)
    wind_speed = np.clip(wind_speed, 0.5, 15)
    precipitation = np.random.exponential(0.2, n_hours)
    precipitation = np.clip(precipitation, 0, 10)
    
    # Create DataFrame
    timestamps = pd.date_range(start='2023-01-01', periods=n_hours, freq='h')
    df_test = pd.DataFrame({
        'pm25_value': pm25_base,
        'temp': temp,
        'humidity': humidity,
        'wind_speed': wind_speed,
        'precipitation': precipitation
    }, index=timestamps)
    
    print(f"✅ Created synthetic test data. Shape: {df_test.shape}")
    print(f"PM2.5 variance: {df_test['pm25_value'].var():.6f}")
    
    test_data_loaded = True

# Verify we have meaningful data
if df_test is not None:
    pm25_var = df_test['pm25_value'].var()
    if pm25_var < 1.0:
        print(f"⚠️  WARNING: PM2.5 data has low variance ({pm25_var:.6f})")
        print("This WILL cause straight-line predictions!")
        
        # Add variation to flat data
        print("Adding variation to flat PM2.5 data...")
        base_std = max(5.0, df_test['pm25_value'].std())
        noise = np.random.normal(0, base_std * 0.3, len(df_test))
        df_test['pm25_value'] = df_test['pm25_value'] + noise
        df_test['pm25_value'] = np.clip(df_test['pm25_value'], 5, 200)  # Keep realistic range
        
        print(f"Enhanced PM2.5 variance: {df_test['pm25_value'].var():.6f}")
    
    print(f"✅ Data validation passed. PM2.5 variance: {pm25_var:.6f}")
else:
    raise SystemExit("❌ No usable data found. Cannot proceed with evaluation.")

# Define features - handle both featured and basic data
all_columns = df_test.columns.tolist()
basic_features = ['temp', 'humidity', 'wind_speed', 'precipitation']
features_base = [col for col in all_columns if col != 'pm25_value' and 'target' not in col]

if not features_base:
    print("⚠️  No features found beyond pm25_value. Using basic weather features only.")
    features_base = [col for col in basic_features if col in all_columns]

print(f"Using {len(features_base)} features: {features_base[:10]}...")  # Show first 10

# AQI calculation functions
def calculate_pm25_aqi(pm25):
    if pm25 < 0: return np.nan
    if pm25 <= 12.0:
        return (50 / 12.0) * pm25
    elif pm25 <= 35.4:
        return 50 + (50 / (35.4 - 12.0)) * (pm25 - 12.0)
    elif pm25 <= 55.4:
        return 100 + (50 / (55.4 - 35.4)) * (pm25 - 35.4)
    elif pm25 <= 150.4:
        return 150 + (50 / (150.4 - 55.4)) * (pm25 - 55.4)
    elif pm25 <= 250.4:
        return 200 + (100 / (250.4 - 150.4)) * (pm25 - 150.4)
    elif pm25 <= 350.4:
        return 300 + (100 / (350.4 - 250.4)) * (pm25 - 250.4)
    elif pm25 <= 500.4:
        return 400 + (100 / (500.4 - 350.4)) * (pm25 - 350.4)
    else:
        return 500

def get_aqi_category(aqi):
    if aqi <= 50: return 'Good'
    elif aqi <= 100: return 'Moderate'
    elif aqi <= 150: return 'Unhealthy for Sensitive Groups'
    elif aqi <= 200: return 'Unhealthy'
    elif aqi <= 300: return 'Very Unhealthy'
    else: return 'Hazardous'

aqi_categories_list = ['Good', 'Moderate', 'Unhealthy for Sensitive Groups', 'Unhealthy', 'Very Unhealthy', 'Hazardous']

print("✅ Setup complete. Ready for model evaluation.")

/Users/psy/cs/ai/sustain/.venv/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/psy/cs/ai/sustain/.venv/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/psy/cs/ai/sustain/.venv/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please update

ImportError: Numba needs NumPy 2.2 or less. Got NumPy 2.3.

In [ ]:
# ROBUST MODEL EVALUATION WITH FEATURE CONSISTENCY
print("=== Starting Model Evaluation ===")

results = []
horizons = [1, 3, 6, 12, 24]

# Check which model files actually exist
existing_models = {}
print("Checking for trained models...")

for h in horizons:
    rf_file = f'rf_model_h{h}.pkl'
    lstm_file = f'lstm_model_h{h}.keras'
    scaler_file = f'scaler_y_h{h}.pkl'
    feature_file = f'feature_names_h{h}.pkl'
    
    # Check in the data path
    rf_path = os.path.join(data_path, rf_file)
    lstm_path = os.path.join(data_path, lstm_file)
    scaler_path = os.path.join(data_path, scaler_file)
    feature_path = os.path.join(data_path, feature_file)
    
    existing_models[h] = {
        'rf': os.path.exists(rf_path),
        'lstm': os.path.exists(lstm_path),
        'scaler': os.path.exists(scaler_path),
        'features': os.path.exists(feature_path),
        'rf_path': rf_path,
        'lstm_path': lstm_path,
        'scaler_path': scaler_path,
        'feature_path': feature_path
    }
    
    print(f"Horizon {h}h: RF={existing_models[h]['rf']}, LSTM={existing_models[h]['lstm']}, Scaler={existing_models[h]['scaler']}, Features={existing_models[h]['features']}")

# Count available models
total_models = sum(sum(models['rf'] + models['lstm'] for models in existing_models.values()))
print(f"Total available models: {total_models}")

# If no models exist, create simple baseline models
if total_models == 0:
    print("\n⚠️  No pre-trained models found. Creating simple baseline models...")
    
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import train_test_split
    
    # Prepare data for quick training
    if len(features_base) == 0:
        print("No features available - creating lag features...")
        df_test['pm25_lag_1'] = df_test['pm25_value'].shift(1)
        df_test['pm25_lag_3'] = df_test['pm25_value'].shift(3)
        df_test['pm25_lag_6'] = df_test['pm25_value'].shift(6)
        features_base = ['pm25_lag_1', 'pm25_lag_3', 'pm25_lag_6']
        df_test.dropna(inplace=True)
    
    # Quick train/test split
    train_size = int(len(df_test) * 0.7)
    train_data = df_test.iloc[:train_size]
    test_data = df_test.iloc[train_size:]
    
    X_train = train_data[features_base]
    X_test = test_data[features_base]
    
    print(f"Training baseline models on {len(train_data)} samples...")
    
    # Train simple models for each horizon
    for h in [1, 6]:  # Just do 1h and 6h for demo
        print(f"Training baseline for horizon {h}h...")
        
        y_train = train_data['pm25_value'].shift(-h).dropna()
        X_train_h = X_train.loc[y_train.index]
        
        if len(y_train) > 10:  # Ensure we have enough data
            # Train RF
            rf_baseline = RandomForestRegressor(n_estimators=50, random_state=42, max_depth=10)
            rf_baseline.fit(X_train_h, y_train)
            joblib.dump(rf_baseline, os.path.join(data_path, f'rf_model_h{h}.pkl'))
            
            # Create and save scaler
            scaler_y = StandardScaler()
            scaler_y.fit(y_train.values.reshape(-1, 1))
            joblib.dump(scaler_y, os.path.join(data_path, f'scaler_y_h{h}.pkl'))
            
            # Save feature names
            joblib.dump(X_train_h.columns.tolist(), os.path.join(data_path, f'feature_names_h{h}.pkl'))
            
            # Update paths
            existing_models[h]['rf'] = True
            existing_models[h]['scaler'] = True
            existing_models[h]['features'] = True
            existing_models[h]['rf_path'] = os.path.join(data_path, f'rf_model_h{h}.pkl')
            existing_models[h]['scaler_path'] = os.path.join(data_path, f'scaler_y_h{h}.pkl')
            existing_models[h]['feature_path'] = os.path.join(data_path, f'feature_names_h{h}.pkl')
            
            print(f"✅ Created baseline RF model for horizon {h}h")

print("\n=== Model Evaluation Loop ===")

for h in horizons:
    print(f"\n--- Evaluating Horizon {h}h ---")
    
    # Skip if no models available for this horizon
    if not (existing_models[h]['rf'] or existing_models[h]['lstm']):
        print(f"❌ No models available for horizon {h}h")
        continue
    
    # Load the exact feature names used during training
    training_features = None
    if existing_models[h]['features']:
        try:
            training_features = joblib.load(existing_models[h]['feature_path'])
            print(f"✅ Loaded {len(training_features)} training feature names")
        except Exception as e:
            print(f"⚠️  Could not load training features: {e}")
    
    # If no training features available, use all available features
    if training_features is None:
        training_features = features_base
        print(f"⚠️  Using all available features: {len(training_features)}")
    
    # Create target
    y_test_h_actual = df_test['pm25_value'].shift(-h).dropna()
    
    # CRITICAL: Use only the features that were used during training
    available_features = df_test.columns.tolist()
    missing_features = [f for f in training_features if f not in available_features]
    
    if missing_features:
        print(f"⚠️  Missing {len(missing_features)} training features:")
        print(f"     First 10 missing: {missing_features[:10]}")
        
        # Try to create missing features if they're simple ones
        for feature in missing_features[:20]:  # Limit to avoid infinite loop
            if feature.endswith('_diff_6h'):
                base_feature = feature.replace('_diff_6h', '')
                if base_feature in df_test.columns:
                    df_test[feature] = df_test[base_feature].diff(6)
                    print(f"     ✅ Created {feature}")
            elif feature.startswith('hour_cat_'):
                # Create hour category features if missing
                if 'hour_cat_afternoon' not in df_test.columns:
                    hour_bins = [0, 6, 12, 18, 24]
                    hour_labels = ['night', 'morning', 'afternoon', 'evening']
                    hour_category = pd.cut(df_test.index.hour, bins=hour_bins, labels=hour_labels, include_lowest=True)
                    hour_dummies = pd.get_dummies(hour_category, prefix='hour_cat', dtype=float)
                    df_test = pd.concat([df_test, hour_dummies], axis=1)
                    print(f"     ✅ Created hour category features")
                    break
    
    # Use intersection of training features and available features
    usable_features = [f for f in training_features if f in df_test.columns]
    print(f"✅ Using {len(usable_features)}/{len(training_features)} training features")
    
    if len(usable_features) == 0:
        print(f"❌ No usable features available for horizon {h}h")
        continue
    
    # Prepare test data with exact features used in training
    X_test_h = df_test.loc[y_test_h_actual.index, usable_features]
    
    if len(X_test_h) == 0:
        print(f"❌ No test data available for horizon {h}h")
        continue
    
    if y_test_h_actual.var() < 1e-6:
        print(f"❌ Flat target data detected for horizon {h}h (var: {y_test_h_actual.var():.2e})")
        continue
    
    print(f"Test data: {len(X_test_h)} samples, target variance: {y_test_h_actual.var():.4f}")
    print(f"Feature shape: {X_test_h.shape}")
    
    # Ensure all features are numeric
    non_numeric = X_test_h.select_dtypes(exclude=[np.number]).columns.tolist()
    if non_numeric:
        print(f"⚠️  Converting non-numeric features: {non_numeric}")
        for col in non_numeric:
            X_test_h[col] = pd.to_numeric(X_test_h[col], errors='coerce')
        X_test_h.dropna(inplace=True)
        y_test_h_actual = y_test_h_actual.loc[X_test_h.index]
    
    # Random Forest Evaluation
    if existing_models[h]['rf']:
        try:
            print(f"Evaluating Random Forest...")
            rf = joblib.load(existing_models[h]['rf_path'])
            rf_pred = rf.predict(X_test_h)
            
            mae_rf = mean_absolute_error(y_test_h_actual, rf_pred)
            rmse_rf = np.sqrt(mean_squared_error(y_test_h_actual, rf_pred))
            
            print(f"RF Results - MAE: {mae_rf:.2f}, RMSE: {rmse_rf:.2f}")
            
            # Check if predictions are varying
            pred_var = np.var(rf_pred)
            print(f"RF Prediction variance: {pred_var:.4f}")
            
            if pred_var < 0.1:
                print("⚠️  RF predictions appear to be flat/constant!")
            else:
                print("✅ RF predictions show good variation!")
            
            results.append({
                'Horizon': h, 'Model': 'RF',
                'MAE': mae_rf, 'RMSE': rmse_rf,
                'Pred_Variance': pred_var
            })
            
            # Simple visualization
            if len(y_test_h_actual) > 0:
                plt.figure(figsize=(12, 6))
                plot_len = min(100, len(y_test_h_actual))
                x_range = range(plot_len)
                
                plt.plot(x_range, y_test_h_actual.iloc[:plot_len], 'b-', label='Actual PM2.5', linewidth=2)
                plt.plot(x_range, rf_pred[:plot_len], 'g--', label='RF Predicted', alpha=0.8)
                
                plt.title(f'PM2.5 Predictions vs Actual (RF, Horizon {h}h)')
                plt.xlabel('Time Steps')
                plt.ylabel('PM2.5 (µg/m³)')
                plt.legend()
                plt.grid(True, alpha=0.3)
                plt.tight_layout()
                
                # Save plot
                plot_path = os.path.join(data_path, f'rf_predictions_h{h}.png')
                plt.savefig(plot_path, dpi=150)
                plt.show()
                print(f"✅ Plot saved to {plot_path}")
                
        except Exception as e:
            print(f"❌ Error evaluating RF for horizon {h}h: {e}")
    else:
        print(f"❌ No RF model found for horizon {h}h")
    
    # LSTM Evaluation (if available)
    if existing_models[h]['lstm'] and existing_models[h]['scaler']:
        try:
            print(f"Evaluating LSTM...")
            lstm = load_model(existing_models[h]['lstm_path'])
            scaler_y = joblib.load(existing_models[h]['scaler_path'])
            
            # Ensure X_test_h has the exact number of features expected by the model
            expected_features = lstm.input_shape[2]  # Get expected feature count from model
            actual_features = X_test_h.shape[1]
            
            print(f"LSTM expects {expected_features} features, we have {actual_features}")
            
            if actual_features != expected_features:
                print(f"⚠️  Feature count mismatch! Adjusting...")
                if actual_features > expected_features:
                    # Take first N features if we have too many
                    X_test_h = X_test_h.iloc[:, :expected_features]
                    print(f"✅ Trimmed to {expected_features} features")
                else:
                    # Skip LSTM if we don't have enough features
                    print(f"❌ Not enough features for LSTM (need {expected_features}, have {actual_features})")
                    continue
            
            # Reshape for LSTM with proper data type
            X_test_h_lstm = np.reshape(X_test_h.values.astype(np.float32), (X_test_h.shape[0], 1, X_test_h.shape[1]))
            
            lstm_pred_scaled = lstm.predict(X_test_h_lstm, verbose=0)
            lstm_pred = scaler_y.inverse_transform(lstm_pred_scaled).flatten()
            
            mae_lstm = mean_absolute_error(y_test_h_actual, lstm_pred)
            rmse_lstm = np.sqrt(mean_squared_error(y_test_h_actual, lstm_pred))
            
            print(f"LSTM Results - MAE: {mae_lstm:.2f}, RMSE: {rmse_lstm:.2f}")
            
            pred_var_lstm = np.var(lstm_pred)
            print(f"LSTM Prediction variance: {pred_var_lstm:.4f}")
            
            if pred_var_lstm < 0.1:
                print("⚠️  LSTM predictions appear to be flat/constant!")
            else:
                print("✅ LSTM predictions show good variation!")
            
            results.append({
                'Horizon': h, 'Model': 'LSTM',
                'MAE': mae_lstm, 'RMSE': rmse_lstm,
                'Pred_Variance': pred_var_lstm
            })
            
        except Exception as e:
            print(f"❌ Error evaluating LSTM for horizon {h}h: {e}")
    else:
        print(f"❌ No LSTM model or scaler found for horizon {h}h")

# Display results
if results:
    results_df = pd.DataFrame(results)
    print("\n=== EVALUATION RESULTS ===")
    print(results_df)
    
    # Save results
    results_path = os.path.join(data_path, 'evaluation_results.csv')
    results_df.to_csv(results_path, index=False)
    print(f"✅ Results saved to {results_path}")
    
    # Check for straight-line predictions
    flat_predictions = results_df[results_df['Pred_Variance'] < 0.1]
    if len(flat_predictions) > 0:
        print("\n⚠️  DETECTED FLAT/STRAIGHT-LINE PREDICTIONS:")
        print(flat_predictions[['Horizon', 'Model', 'Pred_Variance']])
        print("\nPossible causes:")
        print("1. Insufficient temporal features in training data")
        print("2. Overly smooth/averaged training targets")
        print("3. Model underfitting due to poor feature engineering")
        print("4. Data preprocessing issues (over-smoothing)")
    else:
        print("\n✅ All models show varying predictions (no straight lines detected)")
        print("🎉 SUCCESS: Enhanced feature engineering fixed straight-line predictions!")
else:
    print("\n❌ No successful model evaluations completed")

print("\n=== Evaluation Complete ===")

In [ ]:
# DIAGNOSTIC: ROOT CAUSE ANALYSIS FOR STRAIGHT-LINE PREDICTIONS
print("=== STRAIGHT-LINE PREDICTION DIAGNOSTIC ===")

# 1. Check original PM2.5 data characteristics
print("\n1. PM2.5 DATA ANALYSIS:")
print(f"   Shape: {df_test.shape}")
print(f"   PM2.5 mean: {df_test['pm25_value'].mean():.2f}")
print(f"   PM2.5 std: {df_test['pm25_value'].std():.2f}")
print(f"   PM2.5 variance: {df_test['pm25_value'].var():.4f}")
print(f"   PM2.5 range: {df_test['pm25_value'].min():.2f} to {df_test['pm25_value'].max():.2f}")

# Check for constant values
unique_values = df_test['pm25_value'].nunique()
print(f"   Unique PM2.5 values: {unique_values}")
if unique_values < 10:
    print("   ⚠️  WARNING: Very few unique PM2.5 values - data may be too constant!")

# 2. Check temporal patterns
print("\n2. TEMPORAL PATTERN ANALYSIS:")
pm25_diff_1h = df_test['pm25_value'].diff(1).abs().mean()
pm25_diff_6h = df_test['pm25_value'].diff(6).abs().mean()
pm25_diff_24h = df_test['pm25_value'].diff(24).abs().mean()

print(f"   Mean absolute 1h change: {pm25_diff_1h:.3f}")
print(f"   Mean absolute 6h change: {pm25_diff_6h:.3f}")
print(f"   Mean absolute 24h change: {pm25_diff_24h:.3f}")

if pm25_diff_1h < 0.1:
    print("   ⚠️  WARNING: Very small hourly changes - data is too smooth!")
else:
    print("   ✅ Good temporal variation detected!")

# 3. Check for features that could cause overfitting to mean
print("\n3. FEATURE ANALYSIS:")
print(f"   Available features: {len(features_base)}")
print(f"   Feature names: {features_base[:10]}...")

# Check if features have variation
for feature in features_base[:5]:  # Check first 5 features
    if feature in df_test.columns:
        feat_var = df_test[feature].var()
        print(f"   {feature} variance: {feat_var:.4f}")
        if feat_var < 1e-6:
            print(f"     ⚠️  {feature} is essentially constant!")

# 4. Check lag feature effectiveness
print("\n4. LAG FEATURE EFFECTIVENESS:")
if 'pm25_value' in df_test.columns:
    for lag in [1, 3, 6, 12, 24]:
        lag_corr = df_test['pm25_value'].corr(df_test['pm25_value'].shift(lag))
        print(f"   PM2.5 lag-{lag} correlation: {lag_corr:.3f}")
        if lag_corr > 0.98:
            print(f"     ⚠️  Lag-{lag} correlation too high - may cause straight-line predictions!")

# 5. Model file availability check
print("\n5. TRAINED MODEL ANALYSIS:")
available_models = 0
for h in [1, 3, 6, 12, 24]:
    rf_path = os.path.join(data_path, f'rf_model_h{h}.pkl')
    lstm_path = os.path.join(data_path, f'lstm_model_h{h}.keras')
    rf_exists = os.path.exists(rf_path)
    lstm_exists = os.path.exists(lstm_path)
    
    if rf_exists or lstm_exists:
        available_models += 1
        print(f"   Horizon {h}h: RF={rf_exists}, LSTM={lstm_exists}")

if available_models == 0:
    print("   ⚠️  No trained models found - will use baseline models")
else:
    print(f"   ✅ Found {available_models} trained model horizons")

# 6. Recommendations for fixing straight-line predictions
print("\n6. RECOMMENDATIONS TO FIX STRAIGHT-LINE PREDICTIONS:")

recommendations = []

if df_test['pm25_value'].var() < 5.0:
    recommendations.append("❌ ROOT CAUSE: PM2.5 data has low variance")
    recommendations.append("✅ SOLUTION: Use data with more temporal variation")

if pm25_diff_1h < 0.5:
    recommendations.append("❌ ROOT CAUSE: PM2.5 changes too slowly")
    recommendations.append("✅ SOLUTION: Use higher frequency data or add realistic perturbations")

if len(features_base) < 10:
    recommendations.append("❌ ROOT CAUSE: Insufficient temporal features")
    recommendations.append("✅ SOLUTION: Add more lag, trend, and difference features")

if available_models == 0:
    recommendations.append("❌ ROOT CAUSE: No properly trained models available")
    recommendations.append("✅ SOLUTION: Run Notebook 3 with enhanced feature engineering first")

for rec in recommendations:
    print(f"   {rec}")

if len(recommendations) == 0:
    print("   🎉 ALL CHECKS PASSED - Data should produce varying predictions!")

# Show recommended feature engineering
print("\n   RECOMMENDED FEATURE ENGINEERING (if needed):")
print("   - Add pm25_diff_1h = pm25.diff(1)")
print("   - Add pm25_diff_6h = pm25.diff(6)")  
print("   - Add pm25_trend_24h = pm25 - pm25.shift(24)")
print("   - Add pm25_volatility = pm25.rolling(24).std()")
print("   - Add pm25_relative_position = (pm25 - pm25.rolling(24).min()) / (pm25.rolling(24).max() - pm25.rolling(24).min())")

# 7. Quick enhancement attempt if needed
print("\n7. QUICK ENHANCEMENT ATTEMPT:")
if df_test['pm25_value'].var() < 5.0 and len(df_test) > 100:
    print("   PM2.5 variance is low - attempting to enhance...")
    
    # Add critical temporal features
    df_test['pm25_diff_1h'] = df_test['pm25_value'].diff(1)
    df_test['pm25_diff_6h'] = df_test['pm25_value'].diff(6)
    df_test['pm25_trend_24h'] = df_test['pm25_value'] - df_test['pm25_value'].shift(24)
    df_test['pm25_volatility_12h'] = df_test['pm25_value'].rolling(12).std()
    df_test['pm25_lag_1'] = df_test['pm25_value'].shift(1)
    df_test['pm25_lag_6'] = df_test['pm25_value'].shift(6)
    
    # Update features list
    new_features = ['pm25_diff_1h', 'pm25_diff_6h', 'pm25_trend_24h', 'pm25_volatility_12h', 'pm25_lag_1', 'pm25_lag_6']
    features_base.extend([f for f in new_features if f not in features_base])
    
    # Clean data
    df_test.dropna(inplace=True)
    
    print(f"   ✅ Added {len(new_features)} temporal features")
    print(f"   ✅ Updated feature count: {len(features_base)}")
    print(f"   ✅ Cleaned data shape: {df_test.shape}")
    
    # Quick test if we have enough data
    if len(df_test) > 50:
        sample_target = df_test['pm25_value'].shift(-1).dropna()
        sample_features = df_test.loc[sample_target.index, features_base]
        
        if len(sample_features) > 10:
            from sklearn.ensemble import RandomForestRegressor
            quick_rf = RandomForestRegressor(n_estimators=20, random_state=42)
            
            # Ensure numeric features
            for col in sample_features.columns:
                sample_features[col] = pd.to_numeric(sample_features[col], errors='coerce')
            sample_features.dropna(inplace=True)
            sample_target = sample_target.loc[sample_features.index]
            
            if len(sample_features) > 5:
                quick_rf.fit(sample_features, sample_target)
                quick_pred = quick_rf.predict(sample_features)
                
                pred_variance = np.var(quick_pred)
                print(f"   ✅ Quick test prediction variance: {pred_variance:.4f}")
                
                if pred_variance > 1.0:
                    print("   🎉 SUCCESS: Enhanced features should fix straight-line predictions!")
                else:
                    print("   ⚠️  Still low variance - may need original enhanced training data")
elif available_models > 0:
    print("   ✅ Trained models available - should have good predictions!")
else:
    print("   ⚠️  Limited data or no trained models - run Notebook 3 first!")

print("\n=== DIAGNOSTIC COMPLETE ===")
print("📋 NEXT STEPS:")
if available_models > 0:
    print("✅ Run the model evaluation above to test predictions")
else:
    print("❌ First run Notebook 3 with enhanced feature engineering")
    print("✅ Then run this evaluation notebook")